Simple Grid search
- find optimize G
- constant n
- for D days
- with out bet ratio

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplfinance as mpf
import Data_for_bp as dfb
import pickle
from datetime import datetime, timedelta

In [2]:
#import data & count numbers of day
start_date = '15 Aug 2023'
end_date = '16 Aug 2023' # will get data before 17 Aug 2023
# Convert string dates to datetime objects
start_date_dt = datetime.strptime(start_date, '%d %b %Y')
end_date_dt = datetime.strptime(end_date, '%d %b %Y')
# Calculate the number of days (D)
D = (end_date_dt - start_date_dt).days
tick_data = dfb.get_tick_data(start_date,end_date)['EURUSD.mid']

2023-09-18 00:37:44,008 - findatapy.market.datavendorweb - INFO - Request Dukascopy data
2023-09-18 00:37:44,009 - findatapy.market.datavendorweb - INFO - About to download from Dukascopy... for EURUSD
2023-09-18 00:37:44,036 - findatapy.market.datavendorweb - INFO - Downloading... 2023-08-15 00:00:00 https://www.dukascopy.com/datafeed/EURUSD/2023/07/15/00h_ticks.bi5
2023-09-18 00:37:55,160 - findatapy.market.datavendorweb - INFO - Completed request from Dukascopy
2023-09-18 00:37:57,216 - findatapy.market.ioengine - WARNING - Couldn't push MarketDataRequest_858__abstract_curve_key-None__base_depos_currencies-EUR_GBP_AUD_NZD_USD_CAD_CHF_NOK_SEK_JPY__base_depos_tenor-ON_TN_SN_1W_2W_3W_1M_... to Redis: Error 10061 connecting to 127.0.0.1:6379. No connection could be made because the target machine actively refused it.


In [3]:
def get_daily_data(tick_data, d):
    current_date = start_date_dt + timedelta(days=d)
    return tick_data[tick_data.index.date == current_date.date()]

In [4]:
def convert_to_binomial(tick_data,grid_size,ladderized_function):
    ladderized_data = ladderized_function(tick_data,grid_size)
    jumps = dfb.filter_jumps(ladderized_data)
    binomial_data = jumps.diff()/grid_size
    binomial_data = np.vectorize(lambda x: float('{:.5g}'.format(x)))(binomial_data)
    return binomial_data[1:]

def compute_pnl_t(G,n,daily_data,d,position):
    B=dfb.convert_to_binomial(daily_data, G, dfb.ladderize_open)
    pnl = np.zeros(len(B))
    position_series = np.zeros(len(B)) #only store for that day (T period)
    #for t in np.arange(0,len(B)):
    for t in range (len(B)-1):
        if B[t] ==1: 
            position-=n
        elif B[t] ==-1: 
            position+=n
        else:
            position+=0
        pnl[t] = pnl[t-1] + position_series[t-1] * G * B[t-1]
        #pnl[t+1] = position * G * B[t] + pnl[t]

        position_series[t] = position
    #pnl_T= pnl[-1] + position*(daily_data[-1]-G) # cl price - G
    pnl_T= pnl[-1]  #lets drop close price for now
    return pnl,pnl_T,position,position_series 

In [5]:
def compute_pnl_t(G,n,daily_data,d,position):
    B=dfb.convert_to_binomial(daily_data, G, dfb.ladderize_open)
    pnl = np.zeros(len(B))
    position_series = np.zeros(len(B)) ## but for day d > 0 this will not be 0
    for t in np.arange(0,len(B)):
        position_series[t] = position_series[t-1] - n * B[t]
        pnl[t] = pnl[t-1] + position_series[t-1] * G * B[t]
    #pnl_T= pnl[-1] + position*(daily_data[-1]-G) # cl price - G
    pnl_T= pnl[-1] 
    position = position_series[-1]
    return pnl,pnl_T,position,position_series 

In [32]:
pnl,pnl_T,position,position_series = compute_pnl_t(0.00001,1000,tick_data,0,0)

In [33]:
#np.set_printoptions(threshold=np.inf)
print(position_series)
print(pnl)

[      0.     -1000.     -2000.     -3000.     -4000.     -5000.
   -4000.     -5000.     -6000.     -7000.     -8000.     -7000.
   -6000.     -5000.     -4000.     -3000.     -2000.     -1000.
       0.      1000.      2000.      3000.      4000.      5000.
    6000.      5000.      4000.      3000.      2000.      1000.
       0.     -1000.         0.      1000.      2000.      3000.
    4000.      5000.      6000.      7000.      6000.      5000.
    4000.      3000.      4000.      3000.      2000.      1000.
       0.     -1000.     -2000.     -3000.     -4000.     -5000.
   -6000.     -7000.     -8000.     -9000.    -10000.     -9000.
   -8000.     -7000.     -6000.     -5000.     -4000.     -3000.
   -2000.     -1000.         0.      1000.      2000.      3000.
    4000.      5000.      6000.      5000.      6000.      5000.
    4000.      3000.      2000.      1000.         0.     -1000.
   -2000.     -3000.     -4000.     -5000.     -4000.     -5000.
   -4000.     -3000.     

In [15]:
def net_profit(G,n,tick_data):
    profit = np.zeros(D)
    for d in range(D):
        daily_data = get_daily_data(tick_data, d)
        B=dfb.convert_to_binomial(daily_data, G, dfb.ladderize_open)
        if d == 0:
            position = 0
            position_series = np.zeros(len(B)) #only store for that day (T period)
            pnl, pnl_T, position,position_series = compute_pnl_t(G, n, daily_data,d,position)
            profit[d] = pnl_T
        else:
            position_series = position * len(B)
            pnl, pnl_T, position,position_series = compute_pnl_t(G, n, daily_data,d,position)
            profit[d] = pnl_T + profit[d-1]
    net_profit = profit[-1] 
    return net_profit, profit

def net_profit(G,n,tick_data):
    profit = np.zeros(D)
    for d in range(D):
        daily_data = get_daily_data(tick_data, d)
        if d == 0:
            position = 0
            _, pnl_T, position,_ = compute_pnl_t(G, n, daily_data,d,position)
            profit[d] = pnl_T
        else:
            _, pnl_T, position,_ = compute_pnl_t(G, n, daily_data,d,position)
            profit[d] = pnl_T + profit[d-1]
    net_profit = profit[-1] 
    return net_profit, profit

In [16]:
net,profit= net_profit(0.0001,1000,tick_data)
profit

array([94.6397888])

In [21]:
# daily profit > -500K
def profit_constraint(params, n, tick_data):
    G = params[0]
    _, profit = net_profit(G, n, tick_data)
    
    if any(pf < -500000 for pf in profit):
        return -1 #violated
    return 0
# Constraint to ensure -10M < Position at any time t < 10M
def position_constraint(params, n, d):
    G = params[0]
    position = 0
    daily_data = get_daily_data(tick_data, d)
    pnl, _,_, position_series = compute_pnl_t(G, n, daily_data, d, position)
    # Checks if position at any time t violates the constraint
    if any(abs(pos) > 10000000 for pos in position_series):
        return -1 #violated
    return 0

# Constraint to ensure pnl_t at all time t > -150K
def pnl_constraint(params, n, d):
    G = params[0]
    position = 0  # for the first day; may have to modify this
    daily_data = get_daily_data(tick_data, d)
    pnl, _,_, _ = compute_pnl_t(G, n, daily_data, d, position)
    # Checks if pnl at any time t violates the constraint
    if any(p < -150000 for p in pnl):
        return -1
    return 0

In [22]:
def check_constraints(G, n, d):
    position_okay = position_constraint([G], n, d) == 0
    pnl_okay = pnl_constraint([G], n, d) == 0
    profit_okay = profit_constraint([G], n, tick_data) == 0
    return position_okay and pnl_okay and profit_okay

In [ ]:
n=1000 #constant order size

In [23]:
best_profit = float('-inf')
best_G = None

# Custom list of values for G
#G_values = [0.0001, 0.001, 0.01, 0.1, 1]
G_values = [0.0001, 0.001,0.01]
# Grid Search

for G in G_values:
    for d in range(D):
        if check_constraints(G, n, d):
            current_profit,_ = net_profit(G, n, tick_data)
            if current_profit > best_profit:
                best_profit = current_profit
                best_G = G

if best_G is not None:
    print(f"Optimal G: {best_G}")
    print(f"Optimal net profit: {best_profit}")
else:
    print("No feasible solution found.")


Optimal G: 0.0001
Optimal net profit: 94.63978880000002


Check Results

In [17]:
#lets check for accuracy 
n=1000
G = 0.0001
binomial_steps = convert_to_binomial(tick_data,G,dfb.ladderize_open)

array([ 0.      ,  1.      , -1.      ,  1.      , -1.      , -1.      ,
       -1.      ,  1.      ,  1.      , -1.      , -1.      ,  1.      ,
        1.      ,  1.      , -1.      , -1.      , -1.      , -1.      ,
       -1.      ,  1.      , -1.      , -1.      , -1.      ,  1.      ,
        1.      ,  1.      ,  1.      ,  1.      ,  1.      , -1.      ,
       -1.      , -1.      ,  1.      ,  1.      , -1.      , -1.      ,
        1.      ,  1.      ,  1.      , -1.      , -1.      , -1.      ,
       -1.      ,  1.      ,  1.      ,  1.      ,  1.      ,  1.      ,
        1.      ,  1.      ,  1.      , -1.      ,  1.      ,  1.      ,
       -1.      , -1.      , -1.      , -1.      , -1.      , -1.      ,
       -1.      , -1.      , -1.      , -1.      , -1.      , -1.      ,
        1.      ,  1.      ,  1.      ,  1.      ,  1.      ,  1.      ,
        1.      , -1.      ,  1.      ,  1.      ,  1.      ,  1.      ,
        1.      ,  1.      ,  1.      ,  1.      , 

In [18]:
def build_position_sizing(initial_position_sizing,multiplier=1,indicator_data=[]):
    if len(indicator_data) == 0:
        return [initial_position_sizing * (multiplier**i) for i in range(len(binomial_steps))]
    else:
        return initial_position_sizing * indicator_data
def run_strategy_day(binomial_steps,grid_sizing,initial_position_sizing,multiplier=1,indicator_data=[],print_trade_book=False):
    PNL = np.zeros(len(binomial_steps))
    P = np.zeros(len(binomial_steps))
    position_sizing = build_position_sizing(initial_position_sizing,multiplier=1,indicator_data=indicator_data)
    for t in np.arange(0,len(binomial_steps)):
        P[t] = P[t-1] - position_sizing[t] * binomial_steps[t]
        PNL[t] = PNL[t-1] + P[t-1] * grid_sizing * binomial_steps[t]
        if print_trade_book:
            print('t = {}, binary_move={}, PNL = {},  Previous_position= {},current_postion = {}'.format(t,binomial_steps[t],PNL[t],P[t-1],P[t]))
    return PNL,P

In [19]:
PNL, P = run_strategy_day(binomial_steps,G,n,multiplier=1,indicator_data=[],print_trade_book=True)

t = 0, binary_move=0.0, PNL = 0.0,  Previous_position= 0.0,current_postion = 0.0
t = 1, binary_move=1.0, PNL = 0.0,  Previous_position= 0.0,current_postion = -1000.0
t = 2, binary_move=-1.0, PNL = 0.1,  Previous_position= -1000.0,current_postion = 0.0
t = 3, binary_move=1.0, PNL = 0.1,  Previous_position= 0.0,current_postion = -1000.0
t = 4, binary_move=-1.0, PNL = 0.2,  Previous_position= -1000.0,current_postion = 0.0
t = 5, binary_move=-1.0, PNL = 0.2,  Previous_position= 0.0,current_postion = 1000.0
t = 6, binary_move=-1.0, PNL = 0.1,  Previous_position= 1000.0,current_postion = 2000.0
t = 7, binary_move=1.0, PNL = 0.30000000000000004,  Previous_position= 2000.0,current_postion = 1000.0
t = 8, binary_move=1.0, PNL = 0.4,  Previous_position= 1000.0,current_postion = 0.0
t = 9, binary_move=-1.0, PNL = 0.4,  Previous_position= 0.0,current_postion = 1000.0
t = 10, binary_move=-1.0, PNL = 0.30000000000000004,  Previous_position= 1000.0,current_postion = 2000.0
t = 11, binary_move=1.0, PN